In [10]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum
from haversine import haversine, Unit
import time

# data preprocessing

In [11]:
income_df = pd.read_csv('new_income.csv')
employment_df = pd.read_csv('new_employment.csv')
population_df = pd.read_csv('new_population.csv')
facility_df = pd.read_csv('new_child_care.csv')
location_df = pd.read_csv('new_potential_loc.csv')

## (1) disctrits data

In [12]:
merged_df = pd.merge(income_df, employment_df, on = 'zip_code')

def classify_demand(row):
    if row['employment rate'] >= 0.6 or row['average income'] <= 60000:
        return 'High-Demand'
    else:
        return 'Normal-Demand'

merged_df['Demand'] = merged_df.apply(classify_demand, axis=1)

merged_df = pd.merge(merged_df, population_df, on = 'zip_code')

merged_df['p_0-5'] = merged_df['-5']
merged_df['p_5-12'] = np.floor(merged_df['5-9'] + 3/5 * merged_df['10-14'])
merged_df['p_0-12'] = merged_df['p_0-5'] + merged_df['p_5-12']

df = merged_df[['zip_code' , 'Demand', 'p_0-5', 'p_5-12', 'p_0-12']]

df.head()

,zip_code,Demand,p_0-5,p_5-12,p_0-12
0,10001,Normal-Demand,744,1349.0,2093.0
1,10002,High-Demand,2142,4964.0,7106.0
2,10003,Normal-Demand,1440,1605.0,3045.0
3,10004,Normal-Demand,433,278.0,711.0
4,10005,High-Demand,484,341.0,825.0


## (2) facilities data

In [13]:
facility_df['c_0-5'] = facility_df['infant_capacity'] + facility_df['toddler_capacity'] + facility_df['preschool_capacity'] + np.floor(5/12 * facility_df['children_capacity'])
facility_df['c_5-12'] = np.floor(7/12 * facility_df['children_capacity'])
facility_df['c_0-12'] = facility_df['c_0-5'] + facility_df['c_5-12']

facility_df = facility_df[['zip_code' ,'facility_id', 'c_0-5', 'c_5-12', 'c_0-12', 'latitude', 'longitude']]

facility_df.head()

,zip_code,facility_id,c_0-5,c_5-12,c_0-12,latitude,longitude
0,10001,837597,0.0,0.0,0.0,40.748836,-73.999810
1,10001,661697,5.0,7.0,12.0,40.748911,-74.001546
2,10001,837329,0.0,0.0,0.0,40.752093,-74.002588
3,10001,350076,2.0,3.0,5.0,40.748296,-74.001263
4,10001,292419,0.0,0.0,0.0,40.749247,-74.001598


## (3) aggregate of facilities data in every districts

In [14]:
facility = facility_df.groupby('zip_code').agg({
    'c_0-5': 'sum',
    'c_5-12': 'sum',
    'c_0-12': 'sum'
}).reset_index()

facility.head()

,zip_code,c_0-5,c_5-12,c_0-12
0,10001,9.0,13.0,22.0
1,10002,95.0,108.0,203.0
2,10003,0.0,0.0,0.0
3,10004,0.0,0.0,0.0
4,10005,0.0,0.0,0.0


## (4) districts data with current capacity

In [15]:
df = pd.merge(df, facility, on = 'zip_code')
df.head()

,zip_code,Demand,p_0-5,p_5-12,p_0-12,c_0-5,c_5-12,c_0-12
0,10001,Normal-Demand,744,1349.0,2093.0,9.0,13.0,22.0
1,10002,High-Demand,2142,4964.0,7106.0,95.0,108.0,203.0
2,10003,Normal-Demand,1440,1605.0,3045.0,0.0,0.0,0.0
3,10004,Normal-Demand,433,278.0,711.0,0.0,0.0,0.0
4,10005,High-Demand,484,341.0,825.0,0.0,0.0,0.0


## (5) locations data

In [16]:
location_df = location_df.reset_index().rename(columns={'index': 'location_id'})
location_df.head()

,location_id,zip_code,latitude,longitude
0,0,10001,40.741893,-74.000140
1,1,10001,40.752007,-74.005436
2,2,10001,40.750545,-73.997147
3,3,10001,40.744080,-74.001932
4,4,10001,40.748690,-73.999341


# question 1

In [17]:
start_time = time.time()

# Initialize the model
model = Model('ChildCareOptimization')

facility_sizes = {
    'Small': {'TotalSlots': 100, 'Under5Slots': 50, 'Cost': 65000},
    'Medium': {'TotalSlots': 200, 'Under5Slots': 100, 'Cost': 95000},
    'Large': {'TotalSlots': 400, 'Under5Slots': 200, 'Cost': 115000}
}

under5_equipment_cost = 100

I = {}  # Additional slots through expansion at each facility
U = {}  # Under-5 slots added through expansion at each facility
N = {}  # Number of new facilities built in each zip code

# **Initialize the cost components outside the loop**
expansion_costs = []
construction_costs = []

for index, row in df.iterrows():
    z = row['zip_code']
    demand_class = row['Demand']
    population_0_5 = row['p_0-5']
    population_0_12 = row['p_0-12']
    existing_slots_0_5 = row['c_0-5']
    existing_slots_0_12 = row['c_0-12']

    # Get facilities in zip code z
    facilities_in_zip = facility_df[facility_df['zip_code'] == z]

    # Initialize lists to collect variables for constraints per zip code
    expansion_slots = []
    under5_expansion_slots = []

    # Decision variables for expansions at each facility in zip code z
    for idx, facility in facilities_in_zip.iterrows():
        f = facility['facility_id']
        current_capacity = facility['c_0-12']
        current_under5 = facility['c_0-5']

        if current_capacity > 0:

            # Decision variables for expansion at facility f
            I[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f'I_{z}_{f}')
            U[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f'U_{z}_{f}')

            # Constraint: U[z, f] <= I[z, f]
            model.addConstr(I[z, f] - U[z, f] >= 0, name=f'Under5ExpansionLimit_{z}_{f}')
            
            if current_capacity >= 500:
                model.addConstr(I[z, f] == 0)
            else:
                max_capacity = min(500, 1.2*current_capacity)
                model.addConstr(I[z, f] + current_capacity <= max_capacity)
                
            # Collect variables for constraints
            expansion_slots.append(I[z, f])
            under5_expansion_slots.append(U[z, f])

            # **Calculate expansion cost for facility f and add to the list**
            base_cost = 20000 + 200 * current_capacity
            expansion_costs.append(
                (I[z, f] / current_capacity) * base_cost + under5_equipment_cost * U[z, f]
            )

        else:
            # Facility with zero capacity cannot be expanded
            continue

    # Decision variables for new facilities in zip code z
    for s in facility_sizes.keys():
        N[z, s] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f'N_{z}_{s}')

        # **Calculate construction cost for new facilities and add to the list**
        facility_info = facility_sizes[s]
        construction_costs.append(
            N[z, s] * facility_info['Cost']
        )

    # Calculate required slots based on demand classification
    if demand_class == 'High-Demand':
        required_total_slots = 0.5 * population_0_12
    else:
        required_total_slots = (1/3) * population_0_12

    required_under5_slots = (2/3) * population_0_5

    # Total existing slots (including expansions)
    total_existing_slots = existing_slots_0_12
    total_existing_under5_slots = existing_slots_0_5

    # Total expansion slots
    total_expansion_slots = quicksum(expansion_slots)
    total_under5_expansion_slots = quicksum(under5_expansion_slots)

    # Total new slots from new facilities
    total_new_slots = quicksum(N[z, s] * facility_sizes[s]['TotalSlots'] for s in facility_sizes.keys())
    total_new_under5_slots = quicksum(N[z, s] * facility_sizes[s]['Under5Slots'] for s in facility_sizes.keys())

    # Constraint: Total slots requirement
    model.addConstr(
        total_existing_slots + total_expansion_slots + total_new_slots >= required_total_slots,
        name=f'TotalSlotsRequirement_{z}'
    )

    # Constraint: Under-5 slots requirement
    model.addConstr(
        total_existing_under5_slots + total_under5_expansion_slots + total_new_under5_slots >= required_under5_slots,
        name=f'Under5SlotsRequirement_{z}'
    )

# **Set the objective function outside the loop**
model.setObjective(
    quicksum(expansion_costs) + quicksum(construction_costs),
    GRB.MINIMIZE
)

# Solve the model
model.optimize()

end_time = time.time()
print('runtime: ', end_time - start_time)
'''
# Output the results
if model.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    total_cost = model.objVal
    print(f'Total Minimum Cost: ${total_cost:,.2f}\n')

    # Display expansions at each facility
    print('Facility Expansions:')
    for var in model.getVars():
        if 'I_' in var.varName and var.x > 0:
            print(f'{var.varName} = {var.x}')

    # Display under-5 slots added in expansions
    print('\nUnder-5 Slots in Expansions:')
    for var in model.getVars():
        if 'U_' in var.varName and var.x > 0:
            print(f'{var.varName} = {var.x}')

    # Display new facilities to be built
    print('\nNew Facilities to be Built:')
    for var in model.getVars():
        if 'N_' in var.varName and var.x > 0:
            print(f'{var.varName} = {var.x}')
else:
    print('No optimal solution found.')
'''


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 26352 rows, 27375 columns and 66903 nonzeros
Model fingerprint: 0x479f9b82
Variable types: 0 continuous, 27375 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 1e+04]
Found heuristic solution: objective 3.719100e+08
Presolve removed 26088 rows and 26848 columns
Presolve time: 1.48s
Presolved: 264 rows, 527 columns, 1054 nonzeros
Found heuristic solution: objective 3.436745e+08
Variable types: 0 continuous, 527 integer (122 binary)

Root relaxation: objective 3.436299e+08, 317 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

"\n# Output the results\nif model.status == GRB.OPTIMAL:\n    print('Optimal solution found:')\n    total_cost = model.objVal\n    print(f'Total Minimum Cost: ${total_cost:,.2f}\n')\n\n    # Display expansions at each facility\n    print('Facility Expansions:')\n    for var in model.getVars():\n        if 'I_' in var.varName and var.x > 0:\n            print(f'{var.varName} = {var.x}')\n\n    # Display under-5 slots added in expansions\n    print('\nUnder-5 Slots in Expansions:')\n    for var in model.getVars():\n        if 'U_' in var.varName and var.x > 0:\n            print(f'{var.varName} = {var.x}')\n\n    # Display new facilities to be built\n    print('\nNew Facilities to be Built:')\n    for var in model.getVars():\n        if 'N_' in var.varName and var.x > 0:\n            print(f'{var.varName} = {var.x}')\nelse:\n    print('No optimal solution found.')\n"

# question 2

In [18]:
# Start timing
start_time = time.time()

# Set up the optimization model
model = Model('ChildCareOptimization')

facility_sizes = {
    'Small': {'TotalSlots': 100, 'Under5Slots': 50, 'Cost': 65000},
    'Medium': {'TotalSlots': 200, 'Under5Slots': 100, 'Cost': 95000},
    'Large': {'TotalSlots': 400, 'Under5Slots': 200, 'Cost': 115000}
}

under5_equipment_cost = 100
minimum_distance = 0.06  # miles

# Initialize dictionaries to store variables
I = {}      # Total expansion at each facility
I1 = {}     # Expansion in first tier (up to 10%)
I2 = {}     # Expansion in second tier (10% to 15%)
I3 = {}     # Expansion in third tier (15% to 20%)
U = {}      # Under-5 slots added through expansion at each facility
delta1 = {} # Binary variable indicating if Tier 1 is fully utilized
delta2 = {} # Binary variable indicating if Tier 2 is utilized
y = {}      # Binary variables for potential locations and facility sizes

expansion_costs = []
construction_costs = []

# Preprocess distances between facilities

# List to store distance constraints
distance_constraints = []

t1 = time.time()
# Precompute distances between potential locations and existing facilities
print("Preprocessing distances between potential locations and existing facilities...")

for idx_l, row_l in location_df.iterrows():
    l = row_l['location_id']
    loc_l = (row_l['latitude'], row_l['longitude'])
    z_l = row_l['zip_code']

    # Get existing facilities in the same zip code
    facilities_in_zip = facility_df[facility_df['zip_code'] == z_l]
    for idx_e, row_e in facilities_in_zip.iterrows():
        e = row_e['facility_id']
        loc_e = (row_e['latitude'], row_e['longitude'])

        # Calculate distance
        distance = haversine(loc_l, loc_e, unit=Unit.MILES)

        if distance < minimum_distance:
            # Store constraint data
            distance_constraints.append(('existing', l, e, z_l))
            
t2 = time.time()
print('runtime: ',t2-t1)

# Precompute distances between potential locations within the same zip code
print("Preprocessing distances between potential locations...")

# Group potential locations by zip code to limit comparisons
potential_locations = location_df[['location_id', 'latitude', 'longitude', 'zip_code']]

for z in potential_locations['zip_code'].unique():
    locs_in_zip = potential_locations[potential_locations['zip_code'] == z]
    loc_list = locs_in_zip.to_dict('records')
    for i in range(len(loc_list)):
        l1 = loc_list[i]['location_id']
        loc_l1 = (loc_list[i]['latitude'], loc_list[i]['longitude'])
        for j in range(i + 1, len(loc_list)):
            l2 = loc_list[j]['location_id']
            loc_l2 = (loc_list[j]['latitude'], loc_list[j]['longitude'])
            # Calculate distance
            distance = haversine(loc_l1, loc_l2, unit=Unit.MILES)
            if distance < minimum_distance:
                # Store constraint data
                distance_constraints.append(('potential', l1, l2, z))
                
t3 = time.time()
print('runtime: ', t3-t2)

# For each zip code
for index, row in df.iterrows():
    z = row['zip_code']
    demand_class = row['Demand']
    population_0_5 = row['p_0-5']
    population_0_12 = row['p_0-12']
    existing_slots_0_5 = row['c_0-5']
    existing_slots_0_12 = row['c_0-12']

    # Get facilities in zip code z
    facilities_in_zip = facility_df[facility_df['zip_code'] == z]

    # Initialize lists to collect variables and parameters for constraints
    expansion_slots = []
    under5_expansion_slots = []

    # Decision variables for expansions at each facility in zip code z
    for idx_f, facility in facilities_in_zip.iterrows():
        f = facility['facility_id']
        current_capacity = facility['c_0-12']
        current_under5 = facility['c_0-5']

        if current_capacity > 0:
            # Maximum expansions for each tier
            max_I1 = int(0.10 * current_capacity)
            max_I2 = int(0.05 * current_capacity)
            max_I3 = int(0.05 * current_capacity)
            max_expansion = int(min(0.2 * current_capacity, 500))

            # Decision variables for expansion at facility f
            I1[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_I1, name=f'I1_{z}_{f}')
            I2[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_I2, name=f'I2_{z}_{f}')
            I3[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_I3, name=f'I3_{z}_{f}')
            I[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_expansion, name=f'I_{z}_{f}')
            U[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_expansion, name=f'U_{z}_{f}')

            # Binary variables for tier utilization
            delta1[z, f] = model.addVar(vtype=GRB.BINARY, name=f'delta1_{z}_{f}')
            delta2[z, f] = model.addVar(vtype=GRB.BINARY, name=f'delta2_{z}_{f}')

            # Tier limits and cumulative constraints
            # Tier 1
            model.addConstr(I1[z, f] <= max_I1 * delta1[z, f], name=f'Tier1Limit_{z}_{f}')
            model.addConstr(I1[z, f] >= max_I1 * delta1[z, f], name=f'Tier1FullUtilization_{z}_{f}')
            # Tier 2
            model.addConstr(I2[z, f] <= max_I2 * delta1[z, f], name=f'Tier2Limit1_{z}_{f}')
            model.addConstr(I2[z, f] >= max_I2 * delta2[z, f], name=f'Tier2FullUtilization_{z}_{f}')
            # Tier 3
            model.addConstr(I3[z, f] <= max_I3 * delta2[z, f], name=f'Tier3Limit_{z}_{f}')
            # Ensure delta2 <= delta1
            model.addConstr(delta2[z, f] <= delta1[z, f], name=f'DeltaConstraint_{z}_{f}')

            # Total expansion at facility f
            model.addConstr(I1[z, f] + I2[z, f] + I3[z, f] == I[z, f], name=f'TotalExpansion_{z}_{f}')

            # Under-5 slot limit
            model.addConstr(U[z, f] <= I[z, f], name=f'Under5ExpansionLimit_{z}_{f}')

            # Collect variables for constraints
            expansion_slots.append(I[z, f])
            under5_expansion_slots.append(U[z, f])

            # Calculate expansion costs for each tier and add to the list
            base_cost1 = 20000 + 200 * current_capacity
            base_cost2 = 20000 + 400 * current_capacity
            base_cost3 = 20000 + 1000 * current_capacity

            expansion_costs.append(
                (I1[z, f] / current_capacity) * base_cost1 +
                (I2[z, f] / current_capacity) * base_cost2 +
                (I3[z, f] / current_capacity) * base_cost3 +
                under5_equipment_cost * U[z, f]
            )

    # Decision variables for new facilities in zip code z
    # Gather potential locations in zip code z
    locations_in_zip = location_df[location_df['zip_code'] == z]
    location_ids_in_zip = locations_in_zip['location_id'].tolist()

    for idx_l, loc in locations_in_zip.iterrows():
        l = loc['location_id']
        for s in facility_sizes.keys():
            y[l, s] = model.addVar(vtype=GRB.BINARY, name=f'y_{l}_{s}')

            # Construction cost for building at location l of size s
            construction_costs.append(
                y[l, s] * facility_sizes[s]['Cost']
            )

        # One facility size per location constraint
        model.addConstr(quicksum(y[l, s] for s in facility_sizes.keys()) <= 1, name=f'OneSizePerLocation_{l}')

    # Calculate required slots based on demand classification
    if demand_class == 'High-Demand':
        required_total_slots = 0.5 * population_0_12
    else:
        required_total_slots = (1 / 3) * population_0_12

    required_under5_slots = (2 / 3) * population_0_5

    # Total expansion slots
    total_expansion_slots = quicksum(expansion_slots)
    total_under5_expansion_slots = quicksum(under5_expansion_slots)

    # Total new slots from new facilities in zip code z
    total_new_slots = quicksum(y[l, s] * facility_sizes[s]['TotalSlots']
                               for l in location_ids_in_zip for s in facility_sizes.keys())
    total_new_under5_slots = quicksum(y[l, s] * facility_sizes[s]['Under5Slots']
                                      for l in location_ids_in_zip for s in facility_sizes.keys())

    # Capacity Constraints
    model.addConstr(
        existing_slots_0_12 + total_expansion_slots + total_new_slots >= required_total_slots,
        name=f'TotalSlotsRequirement_{z}'
    )

    model.addConstr(
        existing_slots_0_5 + total_under5_expansion_slots + total_new_under5_slots >= required_under5_slots,
        name=f'Under5SlotsRequirement_{z}'
    )

for constraint in distance_constraints:
    if constraint[0] == 'existing':
        _, l, e, z = constraint
        # Cannot build at location l
        for s in facility_sizes.keys():
            model.addConstr(y[l, s] == 0, name=f'DistanceConstraint_Existing_{l}_{e}_{s}')
    elif constraint[0] == 'potential':
        _, l1, l2, z = constraint
        for s in facility_sizes.keys():
            model.addConstr(y[l1, s] + y[l2, s] <= 1, name=f'DistanceConstraint_Potential_{l1}_{l2}_{s}')

# Set the objective function
model.setObjective(
    quicksum(expansion_costs) + quicksum(construction_costs),
    GRB.MINIMIZE
)

# Solve the model
model.optimize()

end_time = time.time()
print("\ntotal runtime: {:.2f} seconds".format(end_time - start_time))

Preprocessing distances between potential locations and existing facilities...
runtime:  182.40003752708435
Preprocessing distances between potential locations...
runtime:  16.184097290039062
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 336468 rows, 391971 columns and 1378488 nonzeros
Model fingerprint: 0xc894e783
Variable types: 0 continuous, 391971 integer (331206 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [1e+00, 9e+01]
  RHS range        [5e-01, 1e+04]
Presolve removed 182578 rows and 152300 columns (presolve time = 5s) ...
Presolve removed 290278 rows and 320729 columns (presolve time = 69s) ...
Presolve removed 290272 rows and 320727 columns
Presolve time: 69.81s
Presolved: 46196 rows, 7